## Summarizer Intergovernmental Panel on Climate Change home using Web Scraping + LLM and summarizing it to three major languages in Nigeria

In [17]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [18]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [19]:
openai = OpenAI()

In [33]:
class Website:
    def __init__(self,url):
        self.url = url
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url,headers = headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        
    

In [34]:
IPCC_website = Website("https://www.ipcc.ch/")
print(IPCC_website.title)
IPCC_website = website("https://www.ipcc.ch/")
print(IPCC_website.text)

IPCC — Intergovernmental Panel on Climate Change
Menu
About
Data
Documentation
Focal Points Portal
Bureau Portal
Library
Links
Help
Languages
عربي
简体中文
Français
Русский
Español
Search
Reports
Synthesis Report
Working Groups
Activities
News
Calendar
Follow
Share
Special and Methodology Reports
2027 IPCC Methodology Report on Inventories for Short-lived Climate Forcers
Special Report on Climate Change and Cities
Global Warming of 1.5°C
Climate Change and Land
2019 Refinement to the 2006 IPCC Guidelines for National Greenhouse Gas Inventories
The Ocean and Cryosphere in a Changing Climate
Sixth Assessment Report
AR6 Synthesis Report: Climate Change 2023
AR6 Climate Change 2022: Impacts, Adaptation and Vulnerability
AR6 Climate Change 2022: Mitigation of Climate Change
AR6 Climate Change 2021: The Physical Science Basis
Fifth Assessment Report
AR5 Synthesis Report: Climate Change 2014
AR5 Climate Change 2013: The Physical Science Basis
AR5 Climate Change 2014: Impacts, Adaptation, and Vuln

In [20]:
# Define  system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an Climate assistant that analyzes the contents of a climate website \
and provides a short summary in English and three Nigeria languages; Yoruba  and Hausa, ignoring text that might be navigation related. \
Respond in markdown."

In [24]:
def get_user_prompt(website):
    user_prompt = f"You are looking at a website on climate change titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt


In [26]:
print(get_user_prompt(IPCC_website))

You are looking at a website on climate change titled IPCC — Intergovernmental Panel on Climate ChangeThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Menu
About
Data
Documentation
Focal Points Portal
Bureau Portal
Library
Links
Help
Languages
عربي
简体中文
Français
Русский
Español
Search
Reports
Synthesis Report
Working Groups
Activities
News
Calendar
Follow
Share
Special and Methodology Reports
2027 IPCC Methodology Report on Inventories for Short-lived Climate Forcers
Special Report on Climate Change and Cities
Global Warming of 1.5°C
Climate Change and Land
2019 Refinement to the 2006 IPCC Guidelines for National Greenhouse Gas Inventories
The Ocean and Cryosphere in a Changing Climate
Sixth Assessment Report
AR6 Synthesis Report: Climate Change 2023
AR6 Climate Change 2022: Impacts, Adaptation and Vulnerability
AR6 Climate Change 2022: Mitigation of Climate Change
AR

In [27]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_user_prompt(IPCC_website)}
    ]

In [28]:
messages_for(IPCC_website)

[{'role': 'system',
  'content': 'You are an Climate assistant that analyzes the contents of a climate website and provides a short summary in English and three Nigeria languages; Yoruba  and Hausa, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website on climate change titled IPCC — Intergovernmental Panel on Climate ChangeThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nMenu\nAbout\nData\nDocumentation\nFocal Points Portal\nBureau Portal\nLibrary\nLinks\nHelp\nLanguages\nعربي\n简体中文\nFrançais\nРусский\nEspañol\nSearch\nReports\nSynthesis Report\nWorking Groups\nActivities\nNews\nCalendar\nFollow\nShare\nSpecial and Methodology Reports\n2027 IPCC Methodology Report on Inventories for Short-lived Climate Forcers\nSpecial Report on Climate Change and Cities\nGlobal Warming of 1.5°C\nClimate

In [35]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [36]:
summarize("https://www.ipcc.ch/")

'# Summary of IPCC Website\n\nThe **Intergovernmental Panel on Climate Change (IPCC)** is a United Nations body responsible for assessing the science related to climate change. It provides policymakers with scientific assessments on climate change implications and potential future risks, as well as strategies for adaptation and mitigation.\n\n## Key Features:\n- **Assessment Reports**: The IPCC produces comprehensive reports that detail the current state of knowledge on climate change, including its risks and socioeconomic impacts.\n- **Working Groups**: Three main working groups focus on different aspects: \n  - Working Group I: The Physical Science Basis\n  - Working Group II: Impacts, Adaptation, and Vulnerability\n  - Working Group III: Mitigation of Climate Change\n- **Special Reports**: Includes significant reports addressing specific topics like global warming and climate impacts on cities.\n\n## Recent Announcements:\n- The IPCC has called for nominations for participants in wo

In [44]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [45]:
display_summary( "https://www.ipcc.ch/")

## Summary of the IPCC Website

The **Intergovernmental Panel on Climate Change (IPCC)** serves as a critical body under the United Nations dedicated to assessing the scientific knowledge regarding climate change, its implications, and potential risks. It provides policy-relevant assessments but does not engage in original research. The IPCC prepares comprehensive reports that include the latest scientific findings, impacts, and mitigation strategies related to climate change.

### Key Components:
- **Reports**: The IPCC publishes assessment reports, special reports, and methodology reports to inform policymakers and the public. These reports are drafted and reviewed in multiple stages to ensure objectivity and transparency.
  
- **Working Groups**: There are three Working Groups focusing on various aspects of climate change:
  - **WG I**: Physical Science Basis
  - **WG II**: Impacts, Adaptation, and Vulnerability
  - **WG III**: Mitigation of Climate Change
  
- **Upcoming Initiatives**: The IPCC is calling for nominations for workshops on engaging diverse knowledge systems and methods of assessment. Additionally, it is looking for authors for the Seventh Assessment Report.

### Recent Announcements:
- **Nominations for Workshops**: Invitation for participants to engage in workshops scheduled for 2026.
- **Obituary**: The passing of Dr. Mannava V.K. Sivakumar, a respected member of the IPCC, was announced.
- **Lead Author Meeting**: Over 100 experts convened for the first lead author meeting related to the 2027 methodology report.

---

## Yoruba Summary

**IPCC** jẹ́ àjọ tó dá lórí àjọ UN tó ń ṣe àyẹ̀wò ìmọ̀ nípa àwọn ìyípadà ìjìnlẹ̀ ayé, àwọn ìpamọ́ rẹ̀, àti ewu to lè jẹ́. Àjọ náà fi àwọn ìwádìí wọn ṣe àfihàn, ṣùgbọ́n kì í ṣe àmúyẹ́ ìwádìí tirẹ̀. IPCC ń kọ́ àwọn iṣẹ́ àṣeyọrí, àwọn ìpèníjà, àti awọn ìwé ìlànà fún imulẹ̀ àwọn gàsì ti ilẹ̀.

### Àwọn kókó pàtàkì:
- **Àwọn Iṣẹ́**: IPCC fi àwọn ìwé àyẹ̀wò, àwọn ìwé àṣeyọrí, àti àwọn ìwé ìlànà tọ́ka sí.
  
- **Àwọn Ẹgbẹ́ Iṣẹ́**: Ẹgbẹ́ mẹta lo wa tí ó ń ṣiṣẹ́ lórí àwọn apá oriṣiriṣi ti ìyípadà ayé.
  
- **Àwọn Ìpèníjà Tó Nbọ**: IPCC n pe fún ìtọ́kasí àwọn olùkópa ṣe nípá ọkọ ayẹwoàpè ní 2026. 

### Àwọn ìkìlọ̀ tuntun:
- **Ìtọ́kasí fún Àwọn Workshop**: Àpèjọ áti ohun èlò yíyọrisi nẹtiwọki.
- **Ikú Dr. Mannava V.K. Sivakumar**: Ikú olóòtú tó jẹ̀ alákóso kan ni IPCC.
- **Ìpàdé Àkọ́kọ́ fún Ìròyìn 2027**: Akẹ́kọ̀ọ́ mẹ́rìndínlọ́gọ́rin kópọ̀ jọ fún ìpàdé ikọ̀ ìkọ̀ wíwà.

---

## Hausa Summary

**IPCC** kungiya ce ta Majalisar Dinkin Duniya da ke nazarin kimiyyar canjin yanayi, illolinsa, da hargitsi da zai iya bayyana. IPCC tana bayar da rahotanni masu ma'ana ga masu yanke hukunci amma ba ta gudanar da bincike na kanta ba. Hakanan tana shirya rahotanni masu zurfi akan canjin yanayi.

### Muhimman Abubuwan:
- **Rahotanni**: IPCC tana fitar da rahotanni na nazari, rahotanni na musamman da rahotanni na hanyoyi gami da sabunta bayanai.

- **Kungiyoyi**: Akwai kungiyoyi guda uku da suka mai da hankali kan bangarori masu zaman kansu na canjin yanayi:
  - **Kungiya ta I**: Asalin Kimiyyar Jiki 
  - **Kungiya ta II**: Illoli, Tausayi, da Jurewa
  - **Kungiya ta III**: Rage juyin halitta

- **Shirye-shiryen Gaba**: IPCC tana neman sunayen mahalarta taron karatu da kuma marubuta don rahoton kimiyyar nazari na bakwai.

### Labarai na kwanan nan:
- **Neman Sunaye don Taruka**: An gayyaci mahalarta domin taron daga 2026.
- **Kasuwa**: Kasuwancin Dr. Mannava V.K. Sivakumar an sanar.
- **Taron Marubuta**: Masana sama da 100 sun taru don taron marubuta na rahoton 2027.